In [1]:
import os
import pandas as pd
import numpy as np
import cv2

# Transfer Learning을 위한 라이브러리
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

In [3]:
# 시각화 라이브러리
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# loading the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# converting it to RGB
x_train = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_train]
x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')

x_test = [cv2.cvtColor(cv2.resize(i, (32,32)), cv2.COLOR_GRAY2BGR) for i in x_test]
x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')

from tensorflow.keras.utils import to_categorical
y_train_one = to_categorical(y_train)
y_test_one = to_categorical(y_test)

11490434/11490434 [==============================] - 1s 0us/step


In [17]:
# input layer를 위한 준비
# 학습데이터
inputShape = x_train[0].shape
inputShape

# output layer를 위한 준비
# one_hot_encoding하기 전의 데이터
# np.unique 10개 나온다. 숫자가 10개 있어서
outputShape = len(np.unique(y_train))
outputShape

# hidden layer를 위한 준비
# weights=None 파라미터는 안 가져오고, 그대로 쓰겠다
# include_top=False top에 있는걸 쓰지 않겠다. input layer를 내거 쓰기 위해서다.
vgg_model = VGG16(weights=None, include_top=False)
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [18]:
# 커스텀 input_shape 생성
keras_input = Input(shape=inputShape, name = 'custom_input')
keras_input.shape

# 커스텀모델 생성 
vgg_model_c = vgg_model(inputs=keras_input)

# Fully Connected Layer 추가 생성 
x = Flatten(name='flatten')(vgg_model_c)
x = Dense(64, activation="relu", name='custom_1')(x)
x = Dense(64, activation="relu", name='custom_2')(x)
x = Dense(outputShape, activation='softmax', name='prediction')(x)

# 커스텀 모델 생성
pretrained_model = Model(inputs=keras_input, outputs=x)
pretrained_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_input (InputLayer)   [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 custom_1 (Dense)            (None, 64)                32832     
                                                                 
 custom_2 (Dense)            (None, 64)                4160      
                                                                 
 prediction (Dense)          (None, 10)                650       
                                                                 
Total params: 14,752,330
Trainable params: 14,752,330
Non-t

In [19]:
pretrained_model.compile(loss="categorical_crossentropy", 
              optimizer= "adam",
              metrics=["accuracy"] )

In [20]:
epochs = 10
pretrained_model.fit(x=x_train,
                     y=y_train_one,
                     epochs=10,
                     validation_data = (x_test, y_test_one))

Epoch 1/10
1875/1875 [==============================] - 216s 111ms/step - loss: 0.4284 - accuracy: 0.8700 - val_loss: 0.1149 - val_accuracy: 0.9773
Epoch 2/10
1875/1875 [==============================] - 213s 114ms/step - loss: 0.1515 - accuracy: 0.9684 - val_loss: 0.0965 - val_accuracy: 0.9808
Epoch 3/10
1875/1875 [==============================] - 211s 112ms/step - loss: 0.1231 - accuracy: 0.9755 - val_loss: 0.1428 - val_accuracy: 0.9739
Epoch 4/10
1875/1875 [==============================] - 211s 113ms/step - loss: 0.0957 - accuracy: 0.9811 - val_loss: 0.0687 - val_accuracy: 0.9855
Epoch 5/10
  64/1875 [>.............................] - ETA: 3:15 - loss: 0.0907 - accuracy: 0.9839

KeyboardInterrupt: 

In [22]:
import tensorflow as tf

with tf.device('/device:GPU:0'):
    epochs = 10
    pretrained_model.fit(x=x_train,
                         y=y_train_one,
                         epochs=10,
                         validation_data=(x_test, y_test_one))

Epoch 1/10
1875/1875 [==============================] - 365s 195ms/step - loss: 0.0913 - accuracy: 0.9821 - val_loss: 0.0806 - val_accuracy: 0.9839
Epoch 2/10
1875/1875 [==============================] - 369s 197ms/step - loss: 0.0811 - accuracy: 0.9846 - val_loss: 0.0579 - val_accuracy: 0.9889
Epoch 3/10
1875/1875 [==============================] - 368s 196ms/step - loss: 0.0888 - accuracy: 0.9817 - val_loss: 0.1589 - val_accuracy: 0.9627
Epoch 4/10
1875/1875 [==============================] - 368s 196ms/step - loss: 0.0928 - accuracy: 0.9815 - val_loss: 0.1050 - val_accuracy: 0.9832
Epoch 5/10
1875/1875 [==============================] - 367s 196ms/step - loss: 0.0861 - accuracy: 0.9822 - val_loss: 0.0439 - val_accuracy: 0.9897
Epoch 6/10
1875/1875 [==============================] - 367s 196ms/step - loss: 0.0814 - accuracy: 0.9836 - val_loss: 0.1003 - val_accuracy: 0.9767
Epoch 7/10
1875/1875 [==============================] - 368s 196ms/step - loss: 0.0694 - accuracy: 0.9845 - val_